<a href="https://colab.research.google.com/github/chongzicbo/nlp-ml-dl-notes/blob/master/%E7%9F%A5%E4%B9%8E%E7%AC%94%E8%AE%B0_Self_Attention%E5%92%8CTransformer%E6%A8%A1%E5%9E%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Transformer模型整体结构

<img src="https://pic4.zhimg.com/80/v2-0c259fb2d439b98de27d877dcd3d1fcb_720w.jpg" width="300">

* 宏观上是一个Encoder-Decoder结构。左边部分为Encoder，右边为Decoder和最后的线性输出层。Encoder和Decoder各有6层。
* 整个的Encoder结构里包含6层。每一层里有两层：一层self-attention层和一层全连接层。这里的self-attention层指的是模型中使用的multi-head-attention,其实就是多个self-attention,每个self-attention可以理解为一个head,多个self-attention就是多头。一个self-attention会输出一个结果z,multi-head-attention的输出结果则是把每一个self-attention的输出结果拼接起来。然后输出给后面的全连接网络。
* encoder每一层的输出作为下一层的输入；最后一层的输出作为Decoder的输入。Multi-head-attention和全连接层后都接一个Add和Norm。
* Decoder部分和Encoder一样也有6层，但是每一个单独的decoder和encoder相比，多了一层Masked Multi-head Attention层。使用mask的原因是因为在预测句子的时候，当前时刻是无法获取到未来时刻的信息的。
* Decoder中的第二层attention层就是一个正常的multi-head attention层。但是这里的QKV来源不同。Q来自于上一个decoder 层的输出，而K，V则来自于encoder的输出。剩下的计算基本相同。
* decoder的最后一层也是全连接层；在decoder的三层中，每层后面都接一个Add和Norm。


## 2. Self-Attention详解

Self-Attention结构如下：

<img src="https://pic2.zhimg.com/80/v2-32eb6aa9e23b79784ed1ca22d3f9abf9_720w.jpg" width="300">

对每个self-attention来说，Q(Query), K(Key), V(Value)三个矩阵均来自同一输入，首先我们要计算Q与K之间的点乘，然后为了防止其结果过大，会除以一个尺度标度$\sqrt{d_k}$ ，其中$d_k$ 为一个query和key向量的维度。再利用Softmax操作将其结果归一化为概率分布，然后再乘以矩阵V就得到权重求和的表示。该操作可以表示为 
$$
Attention(Q,K,V)=softmax(\frac{QK^T}{\sqrt{d_k}})V
$$

<img src="https://pic4.zhimg.com/80/v2-eea2dcbfa49df9fb799ef8e6997260bf_720w.jpg" width="300">

其中，$W^Q,W^K,W^V$是模型训练过程中学习到的参数。


<img src="https://pic4.zhimg.com/80/v2-752c1c91e1b4dbca1b64f59a7e026b9b_720w.jpg" width="300">

而multihead就是我们可以有不同的Q,K,V表示，最后再将其结果结合起来，如下图所示：
<img src="https://pic4.zhimg.com/80/v2-3cd76d3e0d8a20d87dfa586b56cc1ad3_720w.jpg" width="500">

这就是基本的Multi-head-Attention单元，对于encoder来说就是利用这些单元叠加，其中，Q、K、V均来自于前一层encoder的输出，即encoder的每个位置都可以注意到前一层encoder的所有位置。\
对于decoder来讲，与encoder有两个不同之处，一个是第一级的Masked Multi-head,另一个是第二级的Multi-head Attention不仅接受来自前一级的输出，还要接受来自于encoder的输出。


## 3. 其它细节
* position encoding
由于该模型没有recurrence或convolution操作，所以没有明确的关于单词在源句子中 位置的相对或绝对信息，为了让模型学习到位置信息，所以添加了position encoding并将其叠加在word embedding上。
* Add和Norm模块
Add代表了Residual Connection，是为了解决多层神经网络训练困难的问题，通过将前一层的信息无差的传到下一层，可以有效的仅关注差异部分。\
Norm则代表了Layer Normalization，通过对层的激活值的归一化，可以加速模型的训练，使其更快的收敛。
